# **0. Importação das Bibliotecas, Configuração dos Gráficos e Carregamento do DataSet**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de estilo para os gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Carregar o dataset
df = pd.read_csv('df_survey_2024.csv')

# **1. Preparação do Terreno.**


In [ ]:
# Seleção e Limpeza dos Dados
cols_to_select = [
    '2.r_modelo_de_trabalho_atual',
    '2.s_modelo_de_trabalho_ideal',
    '2.k_satisfeito_atualmente',
    '2.t_atitude_em_caso_de_retorno_presencial'
]
df_work = df[cols_to_select].copy()

# Remover linhas com dados essenciais ausentes
df_work.dropna(subset=['2.r_modelo_de_trabalho_atual', '2.s_modelo_de_trabalho_ideal'], inplace=True)

# 2. Engenharia de Dados
# Função para padronizar as categorias de modelo de trabalho
def standardize_model(model_text):
    if '100% remoto' in model_text:
        return '100% Remoto'
    elif 'híbrido' in model_text:
        return 'Híbrido'
    elif '100% presencial' in model_text:
        return '100% Presencial'
    return 'Outro'

df_work['modelo_atual'] = df_work['2.r_modelo_de_trabalho_atual'].apply(standardize_model)
df_work['modelo_ideal'] = df_work['2.s_modelo_de_trabalho_ideal'].apply(standardize_model)

# Converter a coluna de satisfação para formato numérico (1 para Satisfeito, 0 para Não Satisfeito)
df_work['satisfeito_num'] = df_work['2.k_satisfeito_atualmente'].apply(lambda x: 1 if x == True else 0)

print(f"Total de respondentes válidos na análise: {len(df_work)}")
df_work[['modelo_atual', 'modelo_ideal', 'satisfeito_num']].head()

# **2. O Cenário Atual vs. O Cenário Ideal - Quantificando a Lacuna**

In [ ]:
# Calcular as distribuições percentuais
atual_dist = df_work['modelo_atual'].value_counts(normalize=True).sort_index()
ideal_dist = df_work['modelo_ideal'].value_counts(normalize=True).sort_index()

# Combinar os dados para plotagem
dist_df = pd.DataFrame({'Atual': atual_dist, 'Ideal': ideal_dist}).reset_index()
dist_df = dist_df.melt(id_vars='index', var_name='Tipo', value_name='Percentual')
dist_df.rename(columns={'index': 'Modelo de Trabalho'}, inplace=True)

# Plotar o gráfico de barras comparativo
plt.figure(figsize=(12, 7))
ax = sns.barplot(data=dist_df, x='Modelo de Trabalho', y='Percentual', hue='Tipo', palette='viridis')

# Adicionar os rótulos de porcentagem
for p in ax.patches:
    if p.get_height() > 0: # Adiciona esta condição para verificar se a altura é maior que zero
        ax.annotate(f'{p.get_height():.1%}',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center',
                    xytext=(0, 9),
                    textcoords='offset points')

plt.title('Comparativo: Modelo de Trabalho Atual vs. Ideal', fontsize=16, fontweight='bold')
plt.ylabel('Percentual de Profissionais')
plt.xlabel('Modelo de Trabalho')
plt.ylim(0, 0.8)
plt.legend(title='Cenário')
plt.savefig('comparativo_modelos_trabalho.png') # Salva o gráfico
plt.show()

# **3. O Impacto na Satisfação e o Risco de Perder Talentos**




In [ ]:
# 1. Análise de Satisfação
satisfaction_by_model = df_work.groupby('modelo_atual')['satisfeito_num'].mean().sort_values(ascending=False).reset_index()
satisfaction_by_model.rename(columns={'satisfeito_num': 'Taxa de Satisfação'}, inplace=True)

plt.figure(figsize=(10, 6))
ax = sns.barplot(data=satisfaction_by_model, x='modelo_atual', y='Taxa de Satisfação', hue='modelo_atual', palette='plasma')
plt.title('Taxa de Satisfação por Modelo de Trabalho Atual', fontsize=16, fontweight='bold')
plt.xlabel('Modelo de Trabalho')
plt.ylabel('Taxa de Satisfação Média')
plt.ylim(0, 1)

for p in ax.patches:
    ax.annotate(f'{p.get_height():.1%}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 9), textcoords='offset points')

plt.savefig('taxa_satisfacao_modelo_trabalho.png') # Salva o gráfico
plt.show()

In [ ]:
# 2. Análise de Risco de Attrition (Evasão)
df_risk = df_work[df_work['modelo_atual'].isin(['100% Remoto', 'Híbrido'])].copy()
df_risk.dropna(subset=['2.t_atitude_em_caso_de_retorno_presencial'], inplace=True)

# Simplificar as categorias de resposta para a visualização
def standardize_attitude(attitude_text):
    if 'Vou procurar outra oportunidade' in attitude_text:
        return 'Procuraria outro emprego'
    elif 'Vou aceitar e retornar' in attitude_text:
        return 'Aceitaria o retorno'
    else:
        return 'Outro' # Agrupa respostas menos frequentes

df_risk['atitude'] = df_risk['2.t_atitude_em_caso_de_retorno_presencial'].apply(standardize_attitude)
attitude_dist = df_risk['atitude'].value_counts()

# Plotar o gráfico de pizza
plt.figure(figsize=(10, 8))
colors = sns.color_palette('Reds_r', len(attitude_dist))
explode = [0.1 if attitude_dist.index[i] == 'Procuraria outro emprego' else 0 for i in range(len(attitude_dist))]

plt.pie(attitude_dist, labels=attitude_dist.index, autopct='%1.1f%%', startangle=140, colors=colors, explode=explode, textprops={'fontweight': 'bold'})
plt.title('Reação a um Retorno Obrigatório ao Escritório\n(Profissionais em modelo Remoto/Híbrido)', fontsize=16, fontweight='bold')
plt.ylabel('')
plt.savefig('reacao_retorno_obrigatorio.png') # Salva o gráfico
plt.show()

# **4. Um Olhar Detalhado sobre o Desalinhamento**



In [ ]:
# Criar a tabela de contingência
contingency_table = pd.crosstab(df_work['modelo_atual'], df_work['modelo_ideal'])

# Normalizar a tabela por linha para obter as porcentagens baseadas no modelo atual
contingency_percentage = contingency_table.divide(contingency_table.sum(axis=1), axis=0) * 100

# Criar figura e subplots
fig, axes = plt.subplots(2, 1, figsize=(12, 16)) # 2 linhas, 1 coluna

# Heatmap para a tabela de contingência (valores absolutos)
sns.heatmap(contingency_table, annot=True, fmt='d', cmap='YlGnBu', linewidths=.5, ax=axes[0])
axes[0].set_title('Matriz de Preferência: Quantidade', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Modelo de Trabalho Ideal')
axes[0].set_ylabel('Modelo de Trabalho Atual')

# Heatmap para a tabela de contingência (porcentagem)
sns.heatmap(contingency_percentage, annot=True, fmt='.1f', cmap='YlGnBu', linewidths=.5, ax=axes[1])
axes[1].set_title('Matriz de Preferência: Porcentagem (%)', fontsize=16, fontweight='bold')
axes[1].set_xlabel('Modelo de Trabalho Ideal')
axes[1].set_ylabel('Modelo de Trabalho Atual') # Repete o rótulo para clareza

plt.tight_layout() # Ajusta o layout para evitar sobreposição
plt.savefig('matriz_preferencia.png') # Salva o gráfico
plt.show()